# TODO Docstring linting...

In [233]:
from enum import Enum

import numpy as np
import pandas as pd
import plotly.express as px
from bs4 import BeautifulSoup

from utils.colors import bcolors

# Première Exploration

In [234]:
chaos = pd.read_csv("../raw_data/simplified/chaos/idfm_impacts_export_2021_01_01_2021_12_31.csv")

C:\Users\alexf\anaconda3\envs\keolis\lib\site-packages\IPython\core\interactiveshell.py:3251: DtypeWarning:

Columns (12,13) have mixed types.Specify dtype option on import or set low_memory=False.



In [235]:
chaos.shape

(82660, 21)

In [236]:
chaos.head()

,reference,tag_name,cause,publication_start_date,publication_end_date,pt_object_type,pt_object_uri,pt_object_name,severity,titre (title),...,email (email),sms ascenseurs (sms),email ascenseur (email),notification (notification),status,application_start_date,application_end_date,periodicity,created_at,updated_at
0,10_TICE-20210104125401,NaN,perturbation,2021-01-04 12:50:00+01:00,2021-01-04 14:50:00+01:00,line,line:0:400400401:401,Unable to find object,perturbée,Perturbations suite incident circulation,...,"<p>Suite à un accident, les arrêts Les Quincon...",NaN,NaN,"Suite à un accident, les arrêts Les Quinconges...",published,2021-01-04 12:50:00+01:00,2021-01-04 14:50:00+01:00,no,2021-01-04 12:59:06+01:00,NaN
1,10_TICE-20210104125401,NaN,perturbation,2021-01-04 12:50:00+01:00,2021-01-04 14:50:00+01:00,line,line:0:400400403:403,Unable to find object,perturbée,Perturbations suite incident circulation,...,"<p>Suite à un accident, les arrêts Les Quincon...",NaN,NaN,"Suite à un accident, les arrêts Les Quinconges...",published,2021-01-04 12:50:00+01:00,2021-01-04 14:50:00+01:00,no,2021-01-04 12:59:06+01:00,NaN
2,10_TICE-20210113095703,NaN,perturbation,2021-01-13 09:50:00+01:00,2021-01-13 10:30:00+01:00,network,network:0:471,Unable to find object,perturbée,Circulation interrompue gare d'Evry-Courcouronnes,...,"<p>En raison d'un colis suspect, la circulatio...",NaN,NaN,"En raison d'un colis suspect, la circulation d...",published,2021-01-13 09:50:00+01:00,2021-01-13 10:30:00+01:00,no,2021-01-13 10:00:03+01:00,2021-01-13 10:37:06+01:00
3,10_TICE-20210115160120,NaN,travaux,2021-01-15 16:00:00+01:00,2021-04-13 12:00:00+02:00,stop_point,stop_point:0:SP:10:263,Unable to find object,perturbée,Travaux à Evry-Courcouronnes,...,"<p>En raison de travaux, les arrêts Beauvoir, ...",NaN,NaN,"En raison de travaux, les arrêts Beauvoir, Les...",published,2021-01-15 16:10:00+01:00,2021-04-13 12:00:00+02:00,no,2021-01-15 16:03:25+01:00,2021-04-13 12:20:09+02:00
4,10_TICE-20210115160120,NaN,travaux,2021-01-15 16:00:00+01:00,2021-04-13 12:00:00+02:00,stop_point,stop_point:0:SP:10:264,Unable to find object,perturbée,Travaux à Evry-Courcouronnes,...,"<p>En raison de travaux, les arrêts Beauvoir, ...",NaN,NaN,"En raison de travaux, les arrêts Beauvoir, Les...",published,2021-01-15 16:10:00+01:00,2021-04-13 12:00:00+02:00,no,2021-01-15 16:03:25+01:00,2021-04-13 12:20:09+02:00


## Cleaning des colonnes inutiles

On retire les colonnes email, email ascenseur et sms ascenseur.

In [237]:
chaos_clean = chaos[['reference',
                     'tag_name',
                     'cause',
                     'publication_start_date',
                     'publication_end_date',
                     'pt_object_type',
                     'pt_object_uri',
                     'pt_object_name',
                     'severity',
                     'titre (title)',
                     'moteur (web)',
                     'notification (notification)',
                     'status',
                     'application_start_date',
                     'application_end_date',
                     'periodicity',
                     'created_at',
                     ]]

In [238]:
chaos_clean.columns = ['reference',
                       'tag_name',
                       'cause',
                       'publication_start_date',
                       'publication_end_date',
                       'pt_object_type',
                       'pt_object_uri',
                       'pt_object_name',
                       'severity',
                       'titre',
                       'moteur',
                       'notification',
                       'status',
                       'application_start_date',
                       'application_end_date',
                       'periodicity',
                       'created_at',
                       ]

In [239]:
def parse_html_to_text(html):
    soup = BeautifulSoup(html, features="html.parser")

    # kill all script and style elements
    for script in soup(["script", "style"]):
        script.extract()  # rip it out

    # get text
    text = soup.get_text()

    # break into lines and remove leading and trailing space on each
    lines = (line.strip() for line in text.splitlines())
    # break multi-headlines into a line each
    chunks = (phrase.strip() for line in lines for phrase in line.split("  "))
    # drop blank lines
    text = '\n'.join(chunk for chunk in chunks if chunk)
    return text.encode('utf-8').decode('utf-8')


chaos_clean["moteur"] = chaos_clean["moteur"].apply(parse_html_to_text)

C:\Users\alexf\AppData\Local\Temp\ipykernel_5052\315342055.py:20: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



* Cause = perturbation

In [240]:
px.bar(chaos_clean.cause.value_counts())

In [241]:
perturbations = chaos_clean[chaos_clean.cause == "perturbation"]

* Periodicité = no

In [242]:
px.bar(perturbations.periodicity.value_counts())

In [243]:
perturbations_noperiodicity = perturbations[perturbations.periodicity == "no"]

* Titre 

In [244]:
perturbations_noperiodicity["titre"].value_counts().head(10)

Remise en marche d'un ascenseur                     6805
Ligne A : perturbé                                   811
COVID-19 Adaptation des horaires                     386
Ligne B : perturbé                                   344
Manifestations sur la voie publique                  226
Bus RATP, trafic perturbé.                           199
Marne-la-Vallée - reprise du trafic                  196
Marne-la-Vallée                                      196
Prévisions suite mouvement social du 6 septembre     168
Ligne R : Paris Lyon - Montereau différé             167
Name: titre, dtype: int64

* Tag_name != Ascenseur

In [245]:
perturbations_noperiodicity["tag_name"].value_counts()

Actualité    32543
Ascenseur     6812
Name: tag_name, dtype: int64

On peut se concentrer sur les manifestations sur la voie publique. On ne s'intéresse pas aux problèmes d'ascenseurs.

In [246]:
perturbations_noascenseurs = perturbations_noperiodicity[perturbations_noperiodicity.tag_name != "Ascenseur"]

* Sévérité

In [247]:
px.bar(perturbations_noascenseurs.severity.value_counts())

In [248]:
infos = perturbations_noascenseurs[perturbations_noascenseurs.severity == "information"].head()
infos.head()

,reference,tag_name,cause,publication_start_date,publication_end_date,pt_object_type,pt_object_uri,pt_object_name,severity,titre,moteur,notification,status,application_start_date,application_end_date,periodicity,created_at
11,10_TICE-20210116182135,NaN,perturbation,2021-01-16 18:20:00+01:00,2021-01-16 20:20:00+01:00,network,network:0:471,Unable to find object,information,Neige : Prévisions du trafic le 16 Janvier 2021,Suite à l'amélioration des conditions météorol...,NaN,published,2021-01-16 18:20:00+01:00,2021-01-16 20:20:00+01:00,no,2021-01-16 18:23:55+01:00
396,40_STIVO-20210607103829,Actualité,perturbation,2021-06-07 10:30:00+02:00,2021-06-07 21:30:00+02:00,line,line:0:059440038:38,Unable to find object,information,Réseau Stivo - Ligne 38 - Risques de retards à...,Risques de retards à prévoir en raison d'un to...,Réseau Stivo - Ligne 38 - Risques de retards à...,published,2021-06-07 10:30:00+02:00,2021-06-07 21:30:00+02:00,no,2021-06-07 10:40:49+02:00
1911,keolis3-20210211134845,Actualité,perturbation,2021-02-11 13:40:00+01:00,2021-02-11 13:50:00+01:00,network,network:0:285,Unable to find object,information,Keolis Versailles - Test,Test jeudi 11 février 2021,NaN,published,2021-02-11 13:40:00+01:00,2021-02-11 13:50:00+01:00,no,2021-02-11 13:50:32+01:00
2327,keolis4-20210210072333,NaN,perturbation,2021-02-10 07:20:00+01:00,2021-02-10 19:10:00+01:00,line,line:0:287191100:191-100,Unable to find object,information,MERCREDI 10 FÉVRIER 2021 - PERTURBATION NEIGEU...,En raison de l'amélioration des conditions de ...,NaN,published,2021-02-10 07:20:00+01:00,2021-02-10 19:10:00+01:00,no,2021-02-10 07:25:22+01:00
2328,keolis4-20210210072653,Actualité,perturbation,2021-02-10 07:20:00+01:00,2021-02-10 19:10:00+01:00,line,line:0:014014012:12,Unable to find object,information,MERCREDI 10 FÉVRIER 2021 - PERTURBATION NEIGEU...,Reprise progressive de l'ensemble des lignes d...,NaN,published,2021-02-10 07:20:00+01:00,2021-02-10 19:10:00+01:00,no,2021-02-10 07:29:43+01:00


In [249]:
infos["titre"].value_counts()

Neige : Prévisions du trafic le 16 Janvier 2021                                                               1
Réseau Stivo - Ligne 38 - Risques de retards à prévoir entre 18h et 21h lundi 7 juin 2021                     1
Keolis Versailles - Test                                                                                      1
MERCREDI 10 FÉVRIER 2021 - PERTURBATION NEIGEUSE - Ligne 191.100  - Mise à jour le 10 février à 12h00         1
MERCREDI 10 FÉVRIER 2021 - PERTURBATION NEIGEUSE - Keolis Seine Sénart - Mise à jour le 10 février à 12:00    1
Name: titre, dtype: int64

In [250]:
bloquante = perturbations_noascenseurs[perturbations_noascenseurs.severity == "bloquante"]
bloquante.head()

,reference,tag_name,cause,publication_start_date,publication_end_date,pt_object_type,pt_object_uri,pt_object_name,severity,titre,moteur,notification,status,application_start_date,application_end_date,periodicity,created_at
10,10_TICE-20210116133459,NaN,perturbation,2021-01-16 13:30:00+01:00,2021-01-16 23:55:00+01:00,network,network:0:471,Unable to find object,bloquante,Neige : Prévisions du trafic le 16 Janvier 2021,"Compte-tenu des conditions météorologiques, le...",NaN,published,2021-01-16 13:30:00+01:00,2021-01-16 23:55:00+01:00,no,2021-01-16 13:36:30+01:00
31,10_TICE-20210209144337,NaN,perturbation,2021-02-09 14:40:00+01:00,2021-02-09 15:10:00+01:00,line,line:0:400400403:403,Unable to find object,bloquante,Neige : perturbations - Mardi 22 janvier à 10h50,"En raison de l'épisode neigeux, des retards, m...",NaN,published,2021-02-09 14:40:00+01:00,2021-02-09 15:10:00+01:00,no,2021-02-09 15:16:52+01:00
33,10_TICE-20210210045510,NaN,perturbation,2021-02-10 04:50:00+01:00,2021-02-11 04:00:00+01:00,network,network:0:471,Unable to find object,bloquante,Neige/verglas : TRAFIC INTERROMPU,Mise à jour le 10/02 à 12h40Compte-tenu des co...,Compte-tenu des conditions météorologiques et ...,published,2021-02-10 04:50:00+01:00,2021-02-11 04:00:00+01:00,no,2021-02-10 04:59:21+01:00
50,10_TICE-20210510073344,NaN,perturbation,2021-05-10 07:30:00+02:00,2021-05-10 23:59:00+02:00,network,network:0:471,Unable to find object,bloquante,Interruption trafic,Suite à l'agression à l'arme blanche d'un cond...,NaN,published,2021-05-10 07:30:00+02:00,2021-05-10 23:59:00+02:00,no,2021-05-10 07:35:24+02:00
56,10_TICE-20210511080911,NaN,perturbation,2021-05-11 08:00:00+02:00,2021-05-12 17:00:00+02:00,stop_point,stop_point:0:SP:10:147,Unable to find object,bloquante,"Déviation sécurité lignes 402, 420 510 et 510P","Suite à l'agression d'un conducteur, les arrêt...",NaN,published,2021-05-11 08:00:00+02:00,2021-05-12 17:00:00+02:00,no,2021-05-11 08:28:46+02:00


In [251]:
bloquante["titre"].value_counts()

Déviation sécurité lignes 402, 420 510 et 510P        161
Stations fermées                                      154
Episode neigeux lignes RATP<trafic interrompu         129
Ligne T11 : totalement interrompu                      63
Métro 4, travaux de modernisation.                     45
                                                     ... 
Ligne T4 : Gargan et Hôpital de Montf. interrompu       1
Ligne L : La Défense - Bécon-Les-B. interrompu          1
LigneP:Paris Est -Meaux/Château-Thierry interrompu      1
Ligne P : Paris-Meaux bus avancés 1 au 5 février        1
LigneC  Juvisy Dourdan La F. St-Martin d'E ralenti      1
Name: titre, Length: 1054, dtype: int64

On retire lorsque c'st juste une information.

In [252]:
perturbations_noinformations = perturbations_noascenseurs[perturbations_noascenseurs.severity != "information"]
perturbations_noinformations.head()

,reference,tag_name,cause,publication_start_date,publication_end_date,pt_object_type,pt_object_uri,pt_object_name,severity,titre,moteur,notification,status,application_start_date,application_end_date,periodicity,created_at
0,10_TICE-20210104125401,NaN,perturbation,2021-01-04 12:50:00+01:00,2021-01-04 14:50:00+01:00,line,line:0:400400401:401,Unable to find object,perturbée,Perturbations suite incident circulation,"Suite à un accident, les arrêts Les Quinconges...","Suite à un accident, les arrêts Les Quinconges...",published,2021-01-04 12:50:00+01:00,2021-01-04 14:50:00+01:00,no,2021-01-04 12:59:06+01:00
1,10_TICE-20210104125401,NaN,perturbation,2021-01-04 12:50:00+01:00,2021-01-04 14:50:00+01:00,line,line:0:400400403:403,Unable to find object,perturbée,Perturbations suite incident circulation,"Suite à un accident, les arrêts Les Quinconges...","Suite à un accident, les arrêts Les Quinconges...",published,2021-01-04 12:50:00+01:00,2021-01-04 14:50:00+01:00,no,2021-01-04 12:59:06+01:00
2,10_TICE-20210113095703,NaN,perturbation,2021-01-13 09:50:00+01:00,2021-01-13 10:30:00+01:00,network,network:0:471,Unable to find object,perturbée,Circulation interrompue gare d'Evry-Courcouronnes,"En raison d'un colis suspect, la circulation d...","En raison d'un colis suspect, la circulation d...",published,2021-01-13 09:50:00+01:00,2021-01-13 10:30:00+01:00,no,2021-01-13 10:00:03+01:00
10,10_TICE-20210116133459,NaN,perturbation,2021-01-16 13:30:00+01:00,2021-01-16 23:55:00+01:00,network,network:0:471,Unable to find object,bloquante,Neige : Prévisions du trafic le 16 Janvier 2021,"Compte-tenu des conditions météorologiques, le...",NaN,published,2021-01-16 13:30:00+01:00,2021-01-16 23:55:00+01:00,no,2021-01-16 13:36:30+01:00
31,10_TICE-20210209144337,NaN,perturbation,2021-02-09 14:40:00+01:00,2021-02-09 15:10:00+01:00,line,line:0:400400403:403,Unable to find object,bloquante,Neige : perturbations - Mardi 22 janvier à 10h50,"En raison de l'épisode neigeux, des retards, m...",NaN,published,2021-02-09 14:40:00+01:00,2021-02-09 15:10:00+01:00,no,2021-02-09 15:16:52+01:00


## Analyse de données

Le dataset retenu est : perturbations_noinformations

Il s'agit de la BDD chaos à laquelle on a retiré les 3 colonnes email, email ascenseur, sms ascenseur et gardé les lignes intéressantes en terme de sévérité, cause...


In [253]:
chaos_processed = perturbations_noinformations[['reference',
                                                'publication_start_date',
                                                'publication_end_date',
                                                'pt_object_type',
                                                'pt_object_uri',
                                                'pt_object_name',
                                                'severity',
                                                'titre',
                                                'moteur',
                                                'notification',
                                                'application_start_date',
                                                'application_end_date',
                                                'created_at',
                                                ]]

In [254]:
for col in ['publication_start_date', 'publication_end_date', 'application_start_date', 'application_end_date',
            'created_at']:
    chaos_processed[col] = pd.to_datetime(chaos_processed[col])

C:\Users\alexf\AppData\Local\Temp\ipykernel_5052\2855205859.py:3: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [255]:
chaos_processed.reference.value_counts()

SOMAP-20210210102217                               129
SOMAP-20210210091827                               129
SOMAP-20211119220815                                95
SOMAP-20210817144628                                81
optile_4923-721385289                               79
                                                  ... 
sncf_PCNV11744112-db63-11eb-bd65-005056a47b86:1      1
sncf_PCNV117134fe-f9fe-11eb-a859-005056a44da2:1      1
sncf_PCNV11710480-b565-11eb-b304-005056a44da2:1      1
sncf_PCNV116e1b12-6415-11ec-82f0-005056a47b86        1
sncf_PCNV45b376de-b5cd-11eb-aa48-005056a44da2:2      1
Name: reference, Length: 25719, dtype: int64

In [256]:
print(chaos_processed.shape)

(41643, 13)


In [257]:
px.box(chaos_processed.reference.value_counts())

In [258]:
chaos_processed.reference.value_counts().mean()

1.61915315525487

In [259]:
chaos_processed[chaos_processed.reference == "SOMAP-20211119220815"]

,reference,publication_start_date,publication_end_date,pt_object_type,pt_object_uri,pt_object_name,severity,titre,moteur,notification,application_start_date,application_end_date,created_at
82153,SOMAP-20211119220815,2021-11-19 22:00:00+01:00,2021-11-20 20:00:00+01:00,line,line:IDFM:C01072,20,perturbée,"Bus RATP, trafic perturbé ce 20/11.",Manifestation sur la voie publique du samedi 2...,NaN,2021-11-20 14:00:00+01:00,2021-11-20 20:00:00+01:00,2021-11-19 22:28:14+01:00
82154,SOMAP-20211119220815,2021-11-19 22:00:00+01:00,2021-11-20 20:00:00+01:00,line,line:IDFM:C01072,20,perturbée,"Bus RATP, trafic perturbé ce 20/11.",Manifestation sur la voie publique du samedi 2...,NaN,2021-11-20 14:00:00+01:00,2021-11-20 20:00:00+01:00,2021-11-19 22:19:21+01:00
82155,SOMAP-20211119220815,2021-11-19 22:00:00+01:00,2021-11-20 20:00:00+01:00,line,line:IDFM:C01072,20,perturbée,"Bus RATP, trafic perturbé ce 20/11.",Manifestation sur la voie publique du samedi 2...,NaN,2021-11-20 14:00:00+01:00,2021-11-20 20:00:00+01:00,2021-11-19 22:43:23+01:00
82156,SOMAP-20211119220815,2021-11-19 22:00:00+01:00,2021-11-20 20:00:00+01:00,line,line:IDFM:C01073,21,perturbée,"Bus RATP, trafic perturbé ce 20/11.",Manifestation sur la voie publique du samedi 2...,NaN,2021-11-20 14:00:00+01:00,2021-11-20 20:00:00+01:00,2021-11-19 22:23:40+01:00
82157,SOMAP-20211119220815,2021-11-19 22:00:00+01:00,2021-11-20 20:00:00+01:00,line,line:IDFM:C01073,21,perturbée,"Bus RATP, trafic perturbé ce 20/11.",Manifestation sur la voie publique du samedi 2...,NaN,2021-11-20 14:00:00+01:00,2021-11-20 20:00:00+01:00,2021-11-19 22:13:38+01:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...
82243,SOMAP-20211119220815,2021-11-19 22:00:00+01:00,2021-11-20 20:00:00+01:00,line,line:IDFM:C02245,59,perturbée,"Bus RATP, trafic perturbé ce 20/11.",Manifestation sur la voie publique du samedi 2...,NaN,2021-11-20 14:00:00+01:00,2021-11-20 20:00:00+01:00,2021-11-19 22:13:38+01:00
82244,SOMAP-20211119220815,2021-11-19 22:00:00+01:00,2021-11-20 20:00:00+01:00,line,line:IDFM:C02246,71,perturbée,"Bus RATP, trafic perturbé ce 20/11.",Manifestation sur la voie publique du samedi 2...,NaN,2021-11-20 14:00:00+01:00,2021-11-20 20:00:00+01:00,2021-11-19 22:28:14+01:00
82245,SOMAP-20211119220815,2021-11-19 22:00:00+01:00,2021-11-20 20:00:00+01:00,line,line:IDFM:C02246,71,perturbée,"Bus RATP, trafic perturbé ce 20/11.",Manifestation sur la voie publique du samedi 2...,NaN,2021-11-20 14:00:00+01:00,2021-11-20 20:00:00+01:00,2021-11-19 22:19:21+01:00
82246,SOMAP-20211119220815,2021-11-19 22:00:00+01:00,2021-11-20 20:00:00+01:00,line,line:IDFM:C02246,71,perturbée,"Bus RATP, trafic perturbé ce 20/11.",Manifestation sur la voie publique du samedi 2...,NaN,2021-11-20 14:00:00+01:00,2021-11-20 20:00:00+01:00,2021-11-19 22:13:38+01:00


In [260]:
(chaos_processed.publication_start_date - chaos_processed.application_start_date).mean()

Timedelta('-1 days +08:26:10.743150110')

In [ ]:
chaos_shortime = chaos_processed[
    chaos_processed.application_start_date - chaos_processed.publication_start_date < pd.Timedelta(2, unit='h')]
chaos_shortime = chaos_shortime[
    chaos_shortime.application_start_date - chaos_shortime.created_at < pd.Timedelta(2, unit='h')]
chaos_shortime.shape

# TODO supprimer les éléments avec une application date trop loin dans le passé (car prévues en avance ? )

In [262]:
px.bar(chaos_shortime.pt_object_type.value_counts())

In [263]:
chaos_shortime.sample(25)

,reference,publication_start_date,publication_end_date,pt_object_type,pt_object_uri,pt_object_name,severity,titre,moteur,notification,application_start_date,application_end_date,created_at
36169,sncf_PCNV2db8b13a-ea6a-11eb-950f-005056a44da2:2,2021-07-21 22:18:48+02:00,2021-07-22 00:55:30+02:00,line,line:IDFM:C01731,R,perturbée,Ligne R : Melun - Paris Lyon interrompu,Le trafic est interrompu de Melun vers Paris G...,Ligne R : Melun - Paris Lyon interrompu,2021-07-21 22:19:00+02:00,2021-07-22 00:55:00+02:00,2021-07-21 23:30:05+02:00
46560,sncf_PCNV7e51e676-4220-11ec-aa7b-005056a47b86:1,2021-11-10 13:19:39+01:00,2021-11-10 19:30:00+01:00,line,line:IDFM:C01843,T4,perturbée,Ligne T4 : ralenti,Le trafic est ralenti sur l'ensemble de la lig...,Ligne T4 : ralenti,2021-11-10 13:16:00+01:00,2021-11-10 19:00:00+01:00,2021-11-10 17:40:06+01:00
33299,sncf_PCNV13229070-672e-11ec-9698-005056a44da2,2021-12-27 17:00:10+01:00,2021-12-28 12:30:00+01:00,line,line:IDFM:C01730,P,perturbée,Ligne P : Meaux - La Ferté Milon perturbé le 2...,"Le mardi 28 décembre, le trafic est perturbé e...",NaN,2021-12-27 16:57:00+01:00,2021-12-28 12:00:00+01:00,2021-12-28 06:15:08+01:00
36160,sncf_PCNV2d7aa9f8-f9a1-11eb-a0c2-005056a44da2:1,2021-08-10 07:02:49+02:00,2021-08-10 08:40:17+02:00,line,line:IDFM:C01742,A,perturbée,Ligne A : La Défense - Cergy le Haut perturbé,Le trafic est perturbé de La Défense vers Cerg...,NaN,2021-08-10 06:57:00+02:00,2021-08-10 08:30:17+02:00,2021-08-10 08:10:17+02:00
18658,ratp_100232,2021-08-26 15:39:00+02:00,2021-08-26 17:07:14+02:00,line,line:IDFM:C01374,4,perturbée,Métro 4 : Incident voyageur > Trafic perturbé,"Jeudi 26 août 2021 à 15h39, en raison d'un inc...",Métro 4 : Incident voyageur > Trafic perturbé,2021-08-26 15:39:00+02:00,2021-08-26 15:52:14+02:00,2021-08-26 15:55:08+02:00
16370,optile_4923-758425060,2021-11-12 00:00:00+01:00,2021-11-23 19:30:00+01:00,line,line:IDFM:C00854,42,perturbée,RESEAU SENART DEV. L.42-42P-43-43P 62A du 08-2...,Du 08 novembre au 23 novembre 2021 de 7h30 à 1...,RESEAU SENART DEV. L.42-42P-43-43P 62A du 08-2...,2021-11-08 07:30:00+01:00,2021-11-23 19:00:00+01:00,2021-11-12 12:05:04+01:00
49686,sncf_PCNV95a061b0-59de-11ec-9b0c-005056a44da2:1,2021-12-10 18:28:22+01:00,2021-12-10 20:00:00+01:00,line,line:IDFM:C01728,D,perturbée,Ligne D Juvisy - Corbeil-Ess. : suppression,Train supprimé :- Train ROVO départ Juvisy 19...,Ligne D Juvisy - Corbeil-Ess. : suppression,2021-12-10 18:22:00+01:00,2021-12-10 19:30:00+01:00,2021-12-10 18:30:03+01:00
60580,sncf_PCNVdb45616c-f687-11eb-9937-005056a47b86:1,2021-08-06 09:28:10+02:00,2021-08-06 11:30:00+02:00,line,line:IDFM:C01739,J,perturbée,Ligne J : Vernon - Paris St-Lazare ralenti,Le trafic est ralenti entre Vernon - Giverny e...,Ligne J : Vernon - Paris St-Lazare ralenti,2021-08-06 09:26:00+02:00,2021-08-06 11:00:00+02:00,2021-08-06 09:30:04+02:00
77676,SOMAP-20210613100318,2021-06-13 10:00:00+02:00,2021-06-13 19:00:00+02:00,line,line:0:100100094:94,Unable to find object,perturbée,"Manifestations sur voie publique ""La Commune +...",En raison de manifestations sur voie publique ...,NaN,2021-06-13 11:00:00+02:00,2021-06-13 19:00:00+02:00,2021-06-13 10:20:20+02:00
65739,sncf_PCNVfcd78fb8-f5cd-11eb-9357-005056a44da2:1,2021-08-05 11:13:38+02:00,2021-08-05 16:30:00+02:00,line,line:IDFM:C01730,P,perturbée,Ligne P : Esbly - Crécy la Ch. interrompu,Le trafic est interrompu entre Esbly et Crécy-...,Ligne P : Esbly - Crécy la Ch. interrompu,2021-08-05 11:08:00+02:00,2021-08-05 16:00:00+02:00,2021-08-05 11:20:04+02:00


In [264]:
unwanted_entries = "|".join(
    ["mouvement social", "grève", "manifestation", "manifestations", "greve", "travaux", "arrêt de travail",
     "mouvement national", "social", "sociale", 'station fermee', 'suite à un incident les départs',
     'argenteuil boucles de seine', 'les courses non réalisées', 'fermetures nocturnes', 'Suite à un incident,',
     'port du masque est obligatoire', 'Prévisions de trafic', 'fête foraine', 'En raison de la fermeture',
     "(fin d'incident)", "Covid 19", "Stations fermées", "Contre le cancer", "STATIONS FERMEES", "Station fermée",
     "Reprise progressif du trafic", "Reprise du trafic", "Reprise circulation", "réseau de vente", "climatisation",
     "(incident terminé)", "remis en circulation", "ALERTE ENLEVEMENT", "desservi normalement",
     "La composition des trains", "de nouveau fluide", "nouvelle offre de transport", "est terminé",
     "Le trafic reprend", "reprend progressivement", "les arrêts provisoires", "Circulation normale sur nos lignes",
     "réseau fonctionne normalement", "reprennent progressivement",
     "dernières annonces gouvernementales", "TRANSDEV VULAINES"]).lower()

regex_day_range = r'du \d+.{4,12} \d* ?au \d+.{4,12} \d*|du \d+/\d+/?\d+?.{1,10}au ?\d+/\d+/?\d+|\d+ \D{4,12} \d+|le \d+/\d+/\d+'

fixed_day_regex = r"|(lundi|mardi|mercredi|jeudi|vendredi|samedi|dimanche) \d+ (janvier|fevrier|mars|avril|mai|juin|juillet|a\w\wt|septembre|octobre|novembre|décembre)"

regex_day_range_title_mask = chaos_shortime["titre"].str.lower().str.contains(regex_day_range + fixed_day_regex)
print(regex_day_range_title_mask.value_counts())
regex_day_range_moteur_mask = chaos_shortime["moteur"].str.lower().str.contains(regex_day_range + fixed_day_regex)
print(regex_day_range_moteur_mask.value_counts())

keyword_in_title_mask = chaos_shortime["titre"].str.lower().str.contains(unwanted_entries)
keyword_in_moteur_mask = chaos_shortime["moteur"].str.lower().str.contains(unwanted_entries)

message_length_in_moteur_mask = chaos_shortime["moteur"].str.len() < 14

chaos_no_manifestation = chaos_shortime[
    ~(
            keyword_in_moteur_mask | keyword_in_title_mask | regex_day_range_title_mask | regex_day_range_moteur_mask | message_length_in_moteur_mask)]

C:\Users\alexf\AppData\Local\Temp\ipykernel_5052\1953628595.py:18: UserWarning:

This pattern has match groups. To actually get the groups, use str.extract.

C:\Users\alexf\AppData\Local\Temp\ipykernel_5052\1953628595.py:20: UserWarning:

This pattern has match groups. To actually get the groups, use str.extract.



False    24149
True      1373
Name: titre, dtype: int64
False    18531
True      6991
Name: moteur, dtype: int64


C:\Users\alexf\AppData\Local\Temp\ipykernel_5052\1953628595.py:23: UserWarning:

This pattern has match groups. To actually get the groups, use str.extract.

C:\Users\alexf\AppData\Local\Temp\ipykernel_5052\1953628595.py:24: UserWarning:

This pattern has match groups. To actually get the groups, use str.extract.



In [265]:
chaos_no_manifestation

,reference,publication_start_date,publication_end_date,pt_object_type,pt_object_uri,pt_object_name,severity,titre,moteur,notification,application_start_date,application_end_date,created_at
0,10_TICE-20210104125401,2021-01-04 12:50:00+01:00,2021-01-04 14:50:00+01:00,line,line:0:400400401:401,Unable to find object,perturbée,Perturbations suite incident circulation,"Suite à un accident, les arrêts Les Quinconges...","Suite à un accident, les arrêts Les Quinconges...",2021-01-04 12:50:00+01:00,2021-01-04 14:50:00+01:00,2021-01-04 12:59:06+01:00
1,10_TICE-20210104125401,2021-01-04 12:50:00+01:00,2021-01-04 14:50:00+01:00,line,line:0:400400403:403,Unable to find object,perturbée,Perturbations suite incident circulation,"Suite à un accident, les arrêts Les Quinconges...","Suite à un accident, les arrêts Les Quinconges...",2021-01-04 12:50:00+01:00,2021-01-04 14:50:00+01:00,2021-01-04 12:59:06+01:00
2,10_TICE-20210113095703,2021-01-13 09:50:00+01:00,2021-01-13 10:30:00+01:00,network,network:0:471,Unable to find object,perturbée,Circulation interrompue gare d'Evry-Courcouronnes,"En raison d'un colis suspect, la circulation d...","En raison d'un colis suspect, la circulation d...",2021-01-13 09:50:00+01:00,2021-01-13 10:30:00+01:00,2021-01-13 10:00:03+01:00
50,10_TICE-20210510073344,2021-05-10 07:30:00+02:00,2021-05-10 23:59:00+02:00,network,network:0:471,Unable to find object,bloquante,Interruption trafic,Suite à l'agression à l'arme blanche d'un cond...,NaN,2021-05-10 07:30:00+02:00,2021-05-10 23:59:00+02:00,2021-05-10 07:35:24+02:00
56,10_TICE-20210511080911,2021-05-11 08:00:00+02:00,2021-05-12 17:00:00+02:00,stop_point,stop_point:0:SP:10:147,Unable to find object,bloquante,"Déviation sécurité lignes 402, 420 510 et 510P","Suite à l'agression d'un conducteur, les arrêt...",NaN,2021-05-11 08:00:00+02:00,2021-05-12 17:00:00+02:00,2021-05-11 08:28:46+02:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...
82653,SOMAP-20211220092143,2021-12-20 09:20:00+01:00,2021-12-20 11:20:00+01:00,line,line:IDFM:C01373,3,perturbée,Métro 3 : Trafic perturbé<incident d'exploitation,Trafic perturbé en raison d'un incident d'expl...,NaN,2021-12-20 09:20:00+01:00,2021-12-20 11:20:00+01:00,2021-12-20 09:24:46+01:00
82654,SOMAP-20211220134014,2021-12-20 13:40:00+01:00,2021-12-20 15:40:00+01:00,line,line:IDFM:C01728,D,perturbée,Train D : trafic perturbé Creil / Melun • Male...,Trafic perturbé en raison d'un bagage oublié. ...,NaN,2021-12-20 13:40:00+01:00,2021-12-20 15:40:00+01:00,2021-12-20 13:42:46+01:00
82655,SOMAP-20211221085223,2021-12-21 08:50:00+01:00,2021-12-21 10:50:00+01:00,line,line:IDFM:C01374,4,perturbée,Métro 4 : Trafic perturbé<incident d'exploitation,Trafic perturbé en raison d'un incident d'expl...,NaN,2021-12-21 08:50:00+01:00,2021-12-21 10:50:00+01:00,2021-12-21 08:55:04+01:00
82656,SOMAP-20211221085556,2021-12-21 08:50:00+01:00,2021-12-21 10:50:00+01:00,line,line:IDFM:C01373,3,perturbée,Métro 3 : trafic perturbé<incident d'exploitation,Trafic perturbé en raison d'un incident d'expl...,NaN,2021-12-21 08:50:00+01:00,2021-12-21 10:50:00+01:00,2021-12-21 08:58:24+01:00


In [266]:
class IncidentType(Enum):
    METEO = "Meteo"
    VOYAGEUR = "Voyageur"
    BAGAGE_OUBLIE = "Bagage oublié"
    PERSONNES_SUR_LES_VOIES = "Personnes sur les voies"
    SIGNALISATION = "Signalisation"
    PANNE_D_UN_TRAIN = "Train en panne"
    MALVEILLANCE = "Malveillance"
    OBSTACLE_VOIE = "Obstacle voie"
    SIGNAL_D_ALARME = "Signal d'alarme"
    ELECTRIQUE = "Electrique"
    INTERVENTION_TECHNIQUE = "Intervention technique"
    MANAGEMENT = "Management"
    INCENDIE = "Incendie"
    EXPLOITATION = "Exploitation"
    TECHNIQUE = "Technique"


chaos_no_manifestation["class"] = np.nan

mapping_classe = {
    "bagage": IncidentType.BAGAGE_OUBLIE.value,
    "bagage oublié": IncidentType.BAGAGE_OUBLIE.value,
    "colis suspect": IncidentType.BAGAGE_OUBLIE.value,
    "mesure de sécurité": IncidentType.BAGAGE_OUBLIE.value,
    "mesures de sécurité": IncidentType.BAGAGE_OUBLIE.value,
    "signalisation": IncidentType.SIGNALISATION.value,
    "passage à niveau": IncidentType.SIGNALISATION.value,
    "Panne d'aiguillage": IncidentType.SIGNALISATION.value,
    "système d'aiguillage": IncidentType.SIGNALISATION.value,
    "distances de sécurité": IncidentType.SIGNALISATION.value,
    "Choc anormal d'un train": IncidentType.SIGNALISATION.value,
    "gestion des circulations": IncidentType.SIGNALISATION.value,
    "panne d'un système d'aiguillage": IncidentType.SIGNALISATION.value,
    "incident sur les voies de garages": IncidentType.SIGNALISATION.value,
    "Dysfonctionnement d'un passage à niveau": IncidentType.SIGNALISATION.value,
    "météo": IncidentType.METEO.value,
    "meteo": IncidentType.METEO.value,
    "neige": IncidentType.METEO.value,
    "verglas": IncidentType.METEO.value,
    "Tempête": IncidentType.METEO.value,
    "intempéries": IncidentType.METEO.value,
    "climatiques": IncidentType.METEO.value,
    "montée des eaux": IncidentType.METEO.value,
    "Crue de la Seine": IncidentType.METEO.value,
    "manque d’adhérence": IncidentType.METEO.value,
    "Manque d'adhérence": IncidentType.METEO.value,
    "conditions climatiques": IncidentType.METEO.value,
    "conditions climatiques dégradées": IncidentType.METEO.value,
    "alimentation": IncidentType.ELECTRIQUE.value,
    "Panne électrique": IncidentType.ELECTRIQUE.value,
    "alimentation électrque": IncidentType.ELECTRIQUE.value,
    "alimentation electrique": IncidentType.ELECTRIQUE.value,
    "alimentation électrique": IncidentType.ELECTRIQUE.value,
    "Incendie dans un local électrique": IncidentType.ELECTRIQUE.value,
    "restitution tardive": IncidentType.INTERVENTION_TECHNIQUE.value,
    "services techniques": IncidentType.INTERVENTION_TECHNIQUE.value,
    "accident de chantier": IncidentType.INTERVENTION_TECHNIQUE.value,
    "intervention technique": IncidentType.INTERVENTION_TECHNIQUE.value,
    "Fin tardive de chantier": IncidentType.INTERVENTION_TECHNIQUE.value,
    "fin tardive d'un chantier": IncidentType.INTERVENTION_TECHNIQUE.value,
    "intervention des équipes techniques": IncidentType.INTERVENTION_TECHNIQUE.value,
    "panne de train": IncidentType.PANNE_D_UN_TRAIN.value,
    "Panne du train": IncidentType.PANNE_D_UN_TRAIN.value,
    "train en panne": IncidentType.PANNE_D_UN_TRAIN.value,
    "rame stationnée": IncidentType.PANNE_D_UN_TRAIN.value,
    "panne mécanique": IncidentType.PANNE_D_UN_TRAIN.value,
    "avarie matériel": IncidentType.PANNE_D_UN_TRAIN.value,
    "panne d’un tram": IncidentType.PANNE_D_UN_TRAIN.value,
    "panne d¿un train": IncidentType.PANNE_D_UN_TRAIN.value,
    "Rame immobilisée": IncidentType.PANNE_D_UN_TRAIN.value,
    "Panne d'un train": IncidentType.PANNE_D_UN_TRAIN.value,
    "panne d’un train": IncidentType.PANNE_D_UN_TRAIN.value,
    "avarie technique": IncidentType.PANNE_D_UN_TRAIN.value,
    "avarie matérielle": IncidentType.PANNE_D_UN_TRAIN.value,
    "panne d’un tramway": IncidentType.PANNE_D_UN_TRAIN.value,
    "panne sur un train": IncidentType.PANNE_D_UN_TRAIN.value,
    "panne sur le train": IncidentType.PANNE_D_UN_TRAIN.value,
    "Problème de porte": IncidentType.PANNE_D_UN_TRAIN.value,
    "fermeture de porte": IncidentType.PANNE_D_UN_TRAIN.value,
    "fermeture des porte": IncidentType.PANNE_D_UN_TRAIN.value,
    "préparation d'un train": IncidentType.PANNE_D_UN_TRAIN.value,
    "atelier de maintenance": IncidentType.PANNE_D_UN_TRAIN.value,
    "préparation de vos train": IncidentType.PANNE_D_UN_TRAIN.value,
    "difficulté d'acheminement du matériel": IncidentType.PANNE_D_UN_TRAIN.value,
    "arbre": IncidentType.OBSTACLE_VOIE.value,
    "animal": IncidentType.OBSTACLE_VOIE.value,
    "Heurt d": IncidentType.OBSTACLE_VOIE.value,
    "Gêne circulation": IncidentType.OBSTACLE_VOIE.value,
    "accident routier": IncidentType.OBSTACLE_VOIE.value,
    "véhicule bloquant": IncidentType.OBSTACLE_VOIE.value,
    "heurt d’un animal": IncidentType.OBSTACLE_VOIE.value,
    "divagation d’animaux": IncidentType.OBSTACLE_VOIE.value,
    "obstacle sur la voie": IncidentType.OBSTACLE_VOIE.value,
    "véhicule sur les voie": IncidentType.OBSTACLE_VOIE.value,
    "accident de circulation": IncidentType.OBSTACLE_VOIE.value,
    "trafic sur l’autoroute": IncidentType.OBSTACLE_VOIE.value,
    "incident affectant la voie": IncidentType.OBSTACLE_VOIE.value,
    "fumée": IncidentType.INCENDIE.value,
    "incendie": IncidentType.INCENDIE.value,
    "Suspicion de fumée": IncidentType.INCENDIE.value,
    "feu aux abords des voie": IncidentType.INCENDIE.value,
    "suicid": IncidentType.VOYAGEUR.value,
    "agression": IncidentType.VOYAGEUR.value,
    "altercation": IncidentType.VOYAGEUR.value,
    "malaise_voyageur": IncidentType.VOYAGEUR.value,
    "malaise voyageur": IncidentType.VOYAGEUR.value,
    "incident voyageur": IncidentType.VOYAGEUR.value,
    "incidents voyageur": IncidentType.VOYAGEUR.value,
    "Accident de voyageur": IncidentType.VOYAGEUR.value,
    "accident de personne": IncidentType.VOYAGEUR.value,
    "Accident de  personne": IncidentType.VOYAGEUR.value,
    "malaise d'un voyageur": IncidentType.VOYAGEUR.value,
    "services de la police": IncidentType.VOYAGEUR.value,
    "intervention des pompier": IncidentType.VOYAGEUR.value,
    "Intervention de la police": IncidentType.VOYAGEUR.value,
    "accident grave de personne": IncidentType.VOYAGEUR.value,
    "accident grave de voyageur": IncidentType.VOYAGEUR.value,
    "intervention des forces de l’ordre": IncidentType.VOYAGEUR.value,
    "abords des voie": IncidentType.PERSONNES_SUR_LES_VOIES.value,
    "personne sur les voie": IncidentType.PERSONNES_SUR_LES_VOIES.value,
    "présence dans les voie": IncidentType.PERSONNES_SUR_LES_VOIES.value,
    "personnes sur les voie": IncidentType.PERSONNES_SUR_LES_VOIES.value,
    "personne dans les voie": IncidentType.PERSONNES_SUR_LES_VOIES.value,
    "Individus dans les voie": IncidentType.PERSONNES_SUR_LES_VOIES.value,
    "personnes dans les voie": IncidentType.PERSONNES_SUR_LES_VOIES.value,
    "personnes aux abords des voies": IncidentType.PERSONNES_SUR_LES_VOIES.value,
    "bris de barrière": IncidentType.MALVEILLANCE.value,
    "acte de malveillance": IncidentType.MALVEILLANCE.value,
    "vitre cassée sur un train": IncidentType.MALVEILLANCE.value,
    "signal d’alarme": IncidentType.SIGNAL_D_ALARME.value,
    "signal d'alarme": IncidentType.SIGNAL_D_ALARME.value,
    "régulation": IncidentType.MANAGEMENT.value,
    "gestion du trafic": IncidentType.MANAGEMENT.value,
    "ordre de la Police": IncidentType.MANAGEMENT.value,
    "manque de personnel": IncidentType.MANAGEMENT.value,
    "conditions non réunies": IncidentType.MANAGEMENT.value,
    "problème de préparation": IncidentType.MANAGEMENT.value,
    "contrainte de production": IncidentType.MANAGEMENT.value,
    "contraintes de production": IncidentType.MANAGEMENT.value,
    "conditions ne sont pas réunie": IncidentType.MANAGEMENT.value,
    "Contrainte liée à la circulation des train": IncidentType.MANAGEMENT.value,
    "exploitation": IncidentType.EXPLOITATION.value,
    "accident important": IncidentType.EXPLOITATION.value,
    "incident exploitation": IncidentType.EXPLOITATION.value,
    "incident d'exploitation": IncidentType.EXPLOITATION.value,
    "incidents d'exploitation": IncidentType.EXPLOITATION.value,
    "incident technique": IncidentType.TECHNIQUE.value,
    "Trafic interrompu": IncidentType.EXPLOITATION.value,
    "trafic est interrompu": IncidentType.EXPLOITATION.value,
    "interrompu": IncidentType.EXPLOITATION.value,
}

C:\Users\alexf\AppData\Local\Temp\ipykernel_5052\2154654576.py:19: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [267]:
#     unclassified_rows.class()
#     unclassified_rows = chaos_no_manifestation[chaos_no_manifestation["class"].isna()]
#     unclassified_rows[unclassified_rows["titre"].str.contains(mapping)]["class"] = "mapping_classe[mapping]"
#     print(unclassified_rows[unclassified_rows["titre"].str.contains(mapping)]["class"])
def classify_row(row):
    unclassified_elems = []
    for mapping in mapping_classe:
        if mapping.lower() in row["titre"].lower() or mapping.lower() in row["moteur"].lower():
            row["class"] = mapping_classe[mapping]
            break
    else:
        unclassified_elems.append(row["moteur"])

    for elem in pd.Series(unclassified_elems).value_counts().index:
        print(f'No classification found for "{bcolors.OKCYAN + elem + bcolors.ENDC}"')
    return row


chaos_classified = chaos_no_manifestation.apply(classify_row, axis=1)

No classification found for "Le trafic est ralenti entre Paris Gare du Nord et Crépy-en-Valois.Motif : incident voie."
No classification found for "Train supprimé :- JAVA au départ de Corbeil Ess. à 05h16 - Arrivée à Juvisy 05h33Prochain train à circuler :- JAVA au départ de Corbeil Ess. à 05h31 - Arrivée à Juvisy 05h48"
No classification found for "Le trafic est perturbé entre Paris Saint-Lazare et Cergy le Haut via Nanterre Université jusqu'à 20h00.Motif : Maintenance sur les voies 8 et 9 à Paris Saint Lazare.SNCF Transilien"
No classification found for "Train supprimé :- le train DEBO de 07h32 au départ de Juvisy, au passage à Brétigny à 07h50 pour une arrivée à 08h22 à Dourdan la Forêt.Prochain train à circuler :- le train DEBO de 08h01 au départ de Juvisy, au passage à Brétigny à 08h21 pour une arrivée à Dourdan la Forêt à 08h52. Motif : absence de conducteur."
No classification found for "Train supprimé : PUMA de 4h48 au départ de Montargis- Prochain train à circuler : PUMA 05h32

C:\Users\alexf\AppData\Local\Temp\ipykernel_5052\4277212699.py:14: DeprecationWarning:

The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.



No classification found for "Suite à un accident, les arrêts Les Quinconges et La Mare aux Moules ne sont pas desservis jusqu'à nouvel avis."
No classification found for "Suite à un accident, les arrêts Les Quinconges et La Mare aux Moules ne sont pas desservis jusqu'à nouvel avis."
No classification found for "En raison d'une organisation Estivale l'arrêt Félicien Rops ne sera pas desservi le 1er juillet 2021 de 7h00 à 21h00. Veuillez vous reporter aux arrêts MJC et Parking Crété."
No classification found for "En raison d'une marche blanche à Saint-Michel-sur-Orge, les arrêts Rue Berlioz et La Tour ne seront pas desservis ce 21/07/2021 de 18h jusqu'à la fin de la marche. Le terminus et le départ de la ligne se feront aux arrêts Coquelicots (Keolis Meyer)."
No classification found for "En raison de la baisse de la fréquentation constatée dans les transports, les bus de la ligne 439 circulent à des fréquences réduites de 9 h à 16 h,Pour connaitre les horaires : sqybus.fr"
No classificat

In [268]:
chaos_classified["class"].value_counts()

Train en panne             2720
Bagage oublié              2257
Signalisation              1927
Voyageur                   1720
Obstacle voie              1542
Personnes sur les voies    1184
Electrique                  941
Exploitation                721
Meteo                       661
Management                  484
Malveillance                347
Technique                    68
Incendie                     63
Intervention technique       44
Signal d'alarme              22
Name: class, dtype: int64

Train en panne             2720
Bagage oublié              2257
Signalisation              1927
Voyageur                   1720
Obstacle voie              1542
Personnes sur les voies    1184
Electrique                  941
Exploitation                721
Meteo                       661
Management                  484
Malveillance                347
Technique                    68
Incendie                     63
Intervention technique       44
Signal d'alarme              22
Name: class, dtype: int64

In [269]:
chaos_classified["class"].isna().sum()

256

256

In [270]:
chaos_classified[chaos_classified["class"].isna()].titre.value_counts()

Ligne T2 : fermeture entre Charlebourg et Puteaux    6
Ligne A : Vincennes - Marne-la-Vallée C perturbé     6
Station république fermée                            5
Ligne T4 : ralenti                                   4
Ligne L : Paris St-Lazare - Cergy le Haut ralenti    3
                                                    ..
Ligne R : horaires modifiés                          1
Ligne K : perturbé                                   1
Ligne J: Paris - Mantes-la-Jolie perturbé            1
Ligne N Garancières - Dreux : retardé                1
Métro 7 : Divers incidents<trafic perturbé.          1
Name: titre, Length: 196, dtype: int64

Ligne T2 : fermeture entre Charlebourg et Puteaux    6
Ligne A : Vincennes - Marne-la-Vallée C perturbé     6
Station république fermée                            5
Ligne T4 : ralenti                                   4
Ligne L : Paris St-Lazare - Cergy le Haut ralenti    3
                                                    ..
Ligne R : horaires modifiés                          1
Ligne K : perturbé                                   1
Ligne J: Paris - Mantes-la-Jolie perturbé            1
Ligne N Garancières - Dreux : retardé                1
Métro 7 : Divers incidents<trafic perturbé.          1
Name: titre, Length: 196, dtype: int64

In [271]:
print(*list(chaos_classified[chaos_classified["class"].isna()].moteur.value_counts().index), sep="\n\n")

Ligne T2 : fermeture entre Charlebourg et Puteaux

Station république fermée

Le trafic est perturbé de Vincennes vers Marne-la-Vallée Chessy.Motif - Dérangement des installations à Cergy le Haut

Le trafic est perturbé sur l'ensemble de la ligne.Motif : conséquence résiduelle d'un incident ayant affecté la voie entre les gares de Nation et VincennesTRANSILIEN SNCF

la station Gaston Roulaud est fermée au public

19/02/2021 Suite à une rupture de canalisation de gaz, les rues Gabriel de Mortillet et Raymond Vidal sont fermées à la Circulation.La ligne R3 Nord et Sud est desservie uniquement entre le RER et Jaurès via Ermitage jusqu'à la reprise de la circulation.Nous vous remercions de votre compréhension.En savoir plus sur cette information.

Trafic fortement ralenti sur la ligne H jusqu'à 20 h. Interruption entre Pontoise et Creil jusqu'à fin de service. Motif : Panne de plusieurs trains à la suite d’orages exceptionnellement violents ce matin sur la ligne

Le trafic est fortement ra

In [272]:
chaos_classified[chaos_classified["class"].isna()].moteur.value_counts()

Ligne T2 : fermeture entre Charlebourg et Puteaux                                                                                                                                                                                                                                                                                                                                    6
Station république fermée                                                                                                                                                                                                                                                                                                                                                            5
Le trafic est perturbé de Vincennes vers Marne-la-Vallée Chessy.Motif - Dérangement des installations à Cergy le Haut                                                                                                                                     

Ligne T2 : fermeture entre Charlebourg et Puteaux                                                                                                                                                                                                                                                                                                                                    6
Station république fermée                                                                                                                                                                                                                                                                                                                                                            5
Le trafic est perturbé de Vincennes vers Marne-la-Vallée Chessy.Motif - Dérangement des installations à Cergy le Haut                                                                                                                                     

In [273]:
px.bar(chaos_classified["class"].value_counts())

In [274]:
chaos_classified.sample(10)

,reference,publication_start_date,publication_end_date,pt_object_type,pt_object_uri,pt_object_name,severity,titre,moteur,notification,application_start_date,application_end_date,created_at,class
33075,sncf_PCNV115431a4-095f-11ec-aaf6-005056a47b86:1,2021-08-30 08:50:54+02:00,2021-08-30 09:30:54+02:00,line,line:IDFM:C01728,D,perturbée,Ligne D : Villeneuve T Créteil.P non desservis,Les trains ne marquent pas l'arrêt à Villeneuv...,NaN,2021-08-30 08:47:00+02:00,2021-08-30 09:30:00+02:00,2021-08-30 08:55:02+02:00,Bagage oublié
35319,sncf_PCNV25e9c256-fabd-11eb-ba7a-005056a44da2:1,2021-08-11 17:17:20+02:00,2021-08-11 18:30:08+02:00,line,line:IDFM:C01737,H,perturbée,Ligne H : Paris Nord - Luzarches fortement ral...,Le trafic est fortement ralenti entre Paris Ga...,Ligne H : Paris Nord - Luzarches fortement ral...,2021-08-11 17:12:00+02:00,2021-08-11 18:20:08+02:00,2021-08-11 18:00:08+02:00,Signalisation
41456,sncf_PCNV581cfa40-bb02-11eb-ae49-005056a44da2:1,2021-05-22 15:33:48+02:00,2021-05-22 17:30:00+02:00,line,line:0:800:C,Unable to find object,perturbée,Ligne C : Brétigny - Dourdan La F. interrompu,Le trafic est interrompu entre Brétigny et Dou...,Ligne C : Brétigny - Dourdan La F. interrompu,2021-05-22 15:28:00+02:00,2021-05-22 17:00:00+02:00,2021-05-22 15:35:02+02:00,Bagage oublié
48952,sncf_PCNV90f5b972-fe98-11eb-b17a-005056a44da2:1,2021-08-16 15:41:06+02:00,2021-08-16 17:00:00+02:00,line,line:IDFM:C01740,L,perturbée,Ligne L : Cergy le Haut - Paris St-Lazare pert...,Le trafic est perturbé entre Cergy le Haut et ...,NaN,2021-08-16 15:37:00+02:00,2021-08-16 16:30:00+02:00,2021-08-16 15:50:03+02:00,Personnes sur les voies
42540,sncf_PCNV5fc04f56-19f1-11ec-9954-005056a47b86:2,2021-09-20 09:03:22+02:00,2021-09-20 14:00:44+02:00,line,line:IDFM:C01743,B,perturbée,Ligne B : Paris Nord - Aulnay s/s Bois interrompu,Le trafic est interrompu entre Paris Gare du N...,Ligne B : Paris Nord - Aulnay s/s Bois interrompu,2021-09-20 09:14:00+02:00,2021-09-20 14:00:00+02:00,2021-09-20 11:06:01+02:00,Electrique
47002,sncf_PCNV81587900-8666-11eb-966f-005056a47b86:1,2021-03-16 15:47:13+01:00,2021-03-16 17:40:15+01:00,line,line:0:800:R,Unable to find object,bloquante,Ligne R : Melun - Moret-Veneux Sab. interrompu,Le trafic est interrompu entre Melun et Moret ...,Ligne R : Melun - Moret-Veneux Sab. interrompu,2021-03-16 15:42:00+01:00,2021-03-16 17:40:00+01:00,2021-03-16 15:50:02+01:00,Voyageur
54675,sncf_PCNVb7c44b50-6243-11eb-986a-005056a47b86:1,2021-01-29 16:07:32+01:00,2021-01-29 21:00:16+01:00,line,line:0:800:J,Unable to find object,perturbée,Ligne J : St-Lazare - Mantes via Poissy ralenti,Le trafic risque d'être ralenti entre Paris Sa...,NaN,2021-01-29 16:05:00+01:00,2021-01-29 21:00:00+01:00,2021-01-29 17:15:06+01:00,Electrique
37105,sncf_PCNV35e26c66-b1c1-11eb-b304-005056a44da2:1,2021-05-10 20:54:53+02:00,2021-05-10 22:30:00+02:00,line,line:0:800:C,Unable to find object,perturbée,Ligne C : Pontoise - Massy - Palaiseau ralenti,Le trafic est ralenti de Pontoise vers Massy -...,Ligne C : Pontoise - Massy - Palaiseau ralenti,2021-05-10 20:52:00+02:00,2021-05-10 21:50:06+02:00,2021-05-10 21:30:08+02:00,Bagage oublié
63848,sncf_PCNVf12432a8-69c7-11eb-8a07-005056a44da2:1,2021-02-08 05:41:40+01:00,2021-02-08 08:30:00+01:00,line,line:0:800:C,Unable to find object,perturbée,Ligne C Vers.Château - Vers.Chantiers: suppres...,Train supprimé :- Le train CIME prévu au dépar...,Ligne C Vers.Château - Vers.Chantiers: suppres...,2021-02-08 05:36:00+01:00,2021-02-08 08:00:00+01:00,2021-02-08 05:45:01+01:00,Train en panne
37632,sncf_PCNV3a70941e-efc0-11eb-91eb-005056a44da2:1,2021-07-28 18:09:11+02:00,2021-07-28 19:30:08+02:00,line,line:IDFM:C01727,C,perturbée,Ligne C : Paris Austerlitz non desservi,Les trains ne marquent pas l'arrêt à Paris Aus...,NaN,2021-07-28 18:00:00+02:00,2021-07-28 19:30:00+02:00,2021-07-28 18:25:05+02:00,Bagage oublié


,reference,publication_start_date,publication_end_date,pt_object_type,pt_object_uri,pt_object_name,severity,titre,moteur,notification,application_start_date,application_end_date,created_at,class
38583,sncf_PCNV41a7f430-ace9-11eb-bc8b-005056a47b86:1,2021-05-04 16:14:29+02:00,2021-05-04 17:50:00+02:00,line,line:0:800:N,Unable to find object,perturbée,Ligne N : Trappes - Rambouillet interrompu,Le trafic est interrompu entre Trappes et Ramb...,NaN,2021-05-04 16:26:00+02:00,2021-05-04 17:20:00+02:00,2021-05-04 17:05:05+02:00,Bagage oublié
41549,sncf_PCNV58aa2136-87bd-11eb-8c74-005056a44da2:1,2021-03-18 08:41:24+01:00,2021-03-18 10:15:19+01:00,line,line:0:800:R,Unable to find object,perturbée,Ligne R : Paris Lyon - Montargis différé,Le train suivant voit son départ différé de 20...,NaN,2021-03-18 08:38:00+01:00,2021-03-18 10:15:00+01:00,2021-03-18 09:25:04+01:00,Train en panne
56774,sncf_PCNVc4ade1b6-1790-11ec-974c-005056a47b86:1,2021-09-17 10:13:30+02:00,2021-09-17 11:50:00+02:00,line,line:IDFM:C01729,E,perturbée,Ligne E Tournan en Brie - Haussmann : suppression,Le train HAVA suivant est supprimé : - Tour...,NaN,2021-09-17 10:06:00+02:00,2021-09-17 11:20:00+02:00,2021-09-17 10:30:09+02:00,Train en panne
34882,sncf_PCNV21b1d384-f3ad-11eb-9c2e-005056a44da2:1,2021-08-02 18:17:26+02:00,2021-08-02 19:10:32+02:00,line,line:IDFM:C01743,B,perturbée,Ligne B : vers Aéroport CDG2/Mitry-Claye perturbé,Le trafic est perturbé de Châtelet les Halles ...,Ligne B : vers Aéroport CDG2/Mitry-Claye perturbé,2021-08-02 18:09:00+02:00,2021-08-02 19:10:00+02:00,2021-08-02 18:20:04+02:00,Malveillance
47649,sncf_PCNV857254be-9e6e-11eb-ab0f-005056a47b86:3,2021-04-16 06:45:00+02:00,2021-04-16 16:22:22+02:00,line,line:0:810:B,Unable to find object,bloquante,Ligne B : très perturbé,Le trafic est très perturbé sur l'ensemble de ...,Ligne B : très perturbé,2021-04-16 07:33:00+02:00,2021-04-16 15:25:04+02:00,2021-04-16 15:05:05+02:00,Incendie
33742,sncf_PCNV173de184-14ac-11ec-9e97-005056a47b86:2,2021-09-13 18:02:58+02:00,2021-09-13 21:00:00+02:00,line,line:IDFM:C01728,D,perturbée,Ligne D : ortement ralenti,Le trafic est fortement ralenti entre Orry-la-...,NaN,2021-09-13 16:40:00+02:00,2021-09-13 18:26:04+02:00,2021-09-13 18:06:05+02:00,Train en panne
33942,sncf_PCNV18d27e52-e39b-11eb-8fe6-005056a47b86:1,2021-07-13 07:07:01+02:00,2021-07-13 08:30:00+02:00,line,line:IDFM:C01727,C,perturbée,Ligne C : fortement ralenti,Le trafic est fortement ralenti entre Biblioth...,Ligne C : fortement ralenti,2021-07-13 07:01:00+02:00,2021-07-13 08:00:00+02:00,2021-07-13 07:30:03+02:00,Bagage oublié
50069,sncf_PCNV97c71ec6-bcad-11eb-a212-005056a47b86:2,2021-05-24 18:32:09+02:00,2021-05-24 21:00:00+02:00,line,line:0:800:R,Unable to find object,perturbée,LigneR Montereau/Montargis Paris fortement ral...,Le trafic est fortement ralenti entre Monterea...,NaN,2021-05-24 18:26:00+02:00,2021-05-24 20:30:00+02:00,2021-05-24 19:50:08+02:00,Voyageur
43591,sncf_PCNV684f9196-3499-11ec-ad77-005056a44da2:1,2021-10-24 09:09:58+02:00,2021-10-24 10:30:00+02:00,line,line:IDFM:C01742,A,perturbée,Ligne A : perturbé,Le trafic est perturbé de Nanterre Préfecture ...,NaN,2021-10-24 09:04:00+02:00,2021-10-24 10:00:00+02:00,2021-10-24 09:45:09+02:00,Electrique
37347,sncf_PCNV387dfae6-1aa6-11ec-80f1-005056a47b86:1,2021-09-21 08:35:22+02:00,2021-09-21 10:00:00+02:00,line,line:IDFM:C01742,A,perturbée,Ligne A : Vincennes - Cergy / Poissy / St-Germain,Ligne A Le trafic est perturbé de Vincennes ve...,Ligne A : Vincennes - Cergy / Poissy / St-Germain,2021-09-21 08:30:00+02:00,2021-09-21 09:30:00+02:00,2021-09-21 08:40:44+02:00,Signalisation


In [275]:
lines_data = pd.read_csv("../navitia_data/lines.csv", index_col=0)

In [276]:
lines_data.sample(10)

,code,name,id,type
1339,51,Hôpital André Mignot - Saint-Quentin en Yvelin...,line:IDFM:C01873,Bus
652,18,Henri IV - Crécy-la-Chapelle,line:IDFM:C00913,Bus
1811,33,Champ de Courses d'Enghien - Parc des Sources,line:IDFM:C00305,Bus
1328,464,Saint-Quentin en Yvelines - Montigny-le-Breton...,line:IDFM:C01538,Bus
402,184-004,Mairie - Gare Routière,line:IDFM:C01004,Bus
1688,205,Les Murs - Bouray,line:IDFM:C00092,Bus
1606,25,25,line:IDFM:C01046,Bus
1156,FUN,Funiculaire de Montmartre - Gare Haute - Funic...,line:IDFM:C01385,Funiculaire
850,46-1,46-1 DESSERTE Ets SCOLAIRES: PROVINS / NANGIS ...,line:IDFM:C01513,Bus
119,14.1,14.1,line:IDFM:C02168,Bus


,code,name,id,type
1740,R,R,line:IDFM:C01731,Train Transilien
1511,47,47,line:IDFM:C00773,Bus
1377,N,Villeneuve-Saint-Georges - Centre Hospitalier ...,line:IDFM:C00584,Bus
1134,487,Porte de l'Essonne - Marché,line:IDFM:C01710,Bus
530,N44,Gare de l'Est - Garges - Sarcelles,line:IDFM:C01395,Bus
1101,350,Porte de la Chapelle - Aéroport CDG 1 (Termina...,line:IDFM:C01298,Bus
1400,13,Juvisy - Bergeries Saint-Hubert,line:IDFM:C00370,Bus
228,CitéVal S,CitéVal Sud Herblay,line:IDFM:C00481,Bus
340,M,Melun - Rond-Point du Bûcher,line:IDFM:C00885,Bus
933,102,Gambetta - Rosny Bois Perrier,line:IDFM:C01131,Bus


In [277]:
lines_data["label"] = lines_data.type + " " + lines_data.code
lines_data.set_index("id", inplace=True)
mapping_lines = lines_data["label"].to_dict()
print(mapping_lines)
chaos_classified["line_name"] = chaos_classified.pt_object_uri.map(mapping_lines)
chaos_classified.sample(10)

{'line:IDFM:C00563': 'Orlyval, CDG VAL CDG VAL', 'line:IDFM:C00562': 'Bus N1', 'line:IDFM:C01792': 'Bus N2', 'line:IDFM:C01562': 'Bus 91-01', 'line:IDFM:C01563': 'Bus 91-02', 'line:IDFM:C01564': 'Bus 91-03', 'line:IDFM:C01565': 'Bus 91-04', 'line:IDFM:C01566': 'Bus 91-05', 'line:IDFM:C01567': 'Bus 91-06', 'line:IDFM:C01568': 'Bus 91-07', 'line:IDFM:C01569': 'Bus 91-08', 'line:IDFM:C01570': 'Bus 91-09', 'line:IDFM:C01571': 'Bus 91-10', 'line:IDFM:C02201': 'Bus 91-11', 'line:IDFM:C01423': 'Bus 1', 'line:IDFM:C01426': 'Bus 2', 'line:IDFM:C02129': 'Bus 3', 'line:IDFM:C02130': 'Bus 3s', 'line:IDFM:C02137': 'Bus 4', 'line:IDFM:C02138': 'Bus 4s', 'line:IDFM:C01427': 'Bus 5', 'line:IDFM:C02131': 'Bus 6', 'line:IDFM:C02136': 'Bus 7', 'line:IDFM:C02135': 'Bus 7s', 'line:IDFM:C01717': 'Bus 8', 'line:IDFM:C02139': 'Bus 8s', 'line:IDFM:C02133': 'Bus 9', 'line:IDFM:C02132': 'Bus 9s', 'line:IDFM:C00324': 'Bus 1', 'line:IDFM:C00292': 'Bus 1', 'line:IDFM:C00294': 'Bus 2', 'line:IDFM:C01473': 'Bus 2', '

,reference,publication_start_date,publication_end_date,pt_object_type,pt_object_uri,pt_object_name,severity,titre,moteur,notification,application_start_date,application_end_date,created_at,class,line_name
56262,sncf_PCNVc1d1da36-ea0b-11eb-9a1d-005056a47b86:1,2021-07-21 12:08:20+02:00,2021-07-21 15:30:00+02:00,line,line:IDFM:C01730,P,perturbée,Ligne P : Marles en Brie - Coulommiers interrompu,Le trafic est interrompu entre Marles-en-Brie ...,NaN,2021-07-21 12:05:00+02:00,2021-07-21 15:00:00+02:00,2021-07-21 12:10:05+02:00,Obstacle voie,Train Transilien P
58939,sncf_PCNVd14a1c64-65ed-11eb-b8a0-005056a47b86:2,2021-02-03 08:02:43+01:00,2021-02-03 11:00:10+01:00,line,line:0:810:B,Unable to find object,perturbée,Ligne B : fortement perturbé,Le trafic est fortement perturbé sur l'ensembl...,Ligne B : fortement perturbé,2021-02-03 08:00:00+01:00,2021-02-03 10:50:10+01:00,2021-02-03 10:30:12+01:00,Obstacle voie,NaN
45113,sncf_PCNV712f7730-f6ce-11eb-829c-005056a47b86:1,2021-08-06 17:53:23+02:00,2021-08-06 19:30:00+02:00,line,line:IDFM:C01728,D,perturbée,Ligne D: Stade de France - Villeneuve St-G ral...,Le trafic est ralenti entre Stade de France Sa...,Ligne D: Stade de France - Villeneuve St-G ral...,2021-08-06 17:49:00+02:00,2021-08-06 19:00:00+02:00,2021-08-06 17:55:05+02:00,Exploitation,RER D
41814,sncf_PCNV5ab71112-8c9c-11eb-a261-005056a44da2:2,2021-03-24 13:27:48+01:00,2021-03-24 14:00:25+01:00,line,line:0:800:N,Unable to find object,bloquante,Ligne N : Paris Montp. - Vers.Chantiers ralenti,Le trafic est ralenti de Paris Montparnasse ve...,Ligne N : Paris Montp. - Vers.Chantiers ralenti,2021-03-24 13:24:00+01:00,2021-03-24 14:00:09+01:00,2021-03-24 13:40:09+01:00,Bagage oublié,NaN
42636,sncf_PCNV608f7002-0c8a-11ec-8266-005056a47b86:2,2021-09-03 09:16:35+02:00,2021-09-03 10:00:11+02:00,line,line:IDFM:C01728,D,perturbée,Ligne D : Stade de France - Melun/Corbeil. ral...,Le trafic est ralenti entre Stade de France Sa...,NaN,2021-09-03 09:36:00+02:00,2021-09-03 10:00:00+02:00,2021-09-03 09:45:06+02:00,Bagage oublié,RER D
61066,sncf_PCNVdf4c32dc-04aa-11ec-84c2-005056a47b86:1,2021-08-24 09:14:04+02:00,2021-08-24 12:30:00+02:00,line,line:IDFM:C01727,C,perturbée,Ligne C :Paris Aust-Versail.RG/St-Quentin ralenti,Le trafic est fortement ralenti entre Paris Au...,Ligne C :Paris Aust-Versail.RG/St-Quentin ralenti,2021-08-24 09:09:00+02:00,2021-08-24 12:00:00+02:00,2021-08-24 09:15:07+02:00,Train en panne,RER C
33746,sncf_PCNV1742a9e0-495d-11ec-b1d2-005056a44da2:1,2021-11-19 18:21:06+01:00,2021-11-19 22:10:48+01:00,line,line:IDFM:C01728,D,perturbée,Ligne D : Combs la Ville - Melun ralenti,Le trafic est ralenti de Combs-la-Ville - Quin...,Ligne D : Combs la Ville - Melun ralenti,2021-11-19 18:17:00+01:00,2021-11-19 22:10:00+01:00,2021-11-19 19:35:05+01:00,Train en panne,RER D
38243,sncf_PCNV3fcdd2da-e01d-11eb-95b4-005056a44da2:1,2021-07-08 20:24:00+02:00,2021-07-08 22:30:00+02:00,line,line:IDFM:C01739,J,perturbée,Ligne J : Boissy l'A. - Gisors interrompu,Le trafic est interrompu entre Boissy-l'Ailler...,NaN,2021-07-08 20:16:00+02:00,2021-07-08 21:10:04+02:00,2021-07-08 20:50:04+02:00,Signalisation,Train Transilien J
35132,sncf_PCNV2454c4bc-9222-11eb-a59d-005056a47b86:1,2021-03-31 15:08:01+02:00,2021-03-31 20:00:10+02:00,line,line:0:800:N,Unable to find object,perturbée,Ligne N : Trafic ralenti.,Le trafic est ralenti sur l'ensemble de la lig...,NaN,2021-03-31 14:56:00+02:00,2021-03-31 20:00:00+02:00,2021-03-31 18:10:11+02:00,Obstacle voie,NaN
33586,sncf_PCNV163b5652-1b70-11ec-a645-005056a47b86:1,2021-09-22 08:18:57+02:00,2021-09-23 09:52:17+02:00,line,line:IDFM:C01731,R,bloquante,Ligne R : totalement interrompu,Le trafic est interrompu sur l'ensemble de la ...,Ligne R : totalement interrompu,2021-09-22 08:38:00+02:00,2021-09-23 09:42:18+02:00,2021-09-23 09:22:42+02:00,Bagage oublié,Train Transilien R


{'line:IDFM:C00563': 'Orlyval, CDG VAL CDG VAL', 'line:IDFM:C00562': 'Bus N1', 'line:IDFM:C01792': 'Bus N2', 'line:IDFM:C01562': 'Bus 91-01', 'line:IDFM:C01563': 'Bus 91-02', 'line:IDFM:C01564': 'Bus 91-03', 'line:IDFM:C01565': 'Bus 91-04', 'line:IDFM:C01566': 'Bus 91-05', 'line:IDFM:C01567': 'Bus 91-06', 'line:IDFM:C01568': 'Bus 91-07', 'line:IDFM:C01569': 'Bus 91-08', 'line:IDFM:C01570': 'Bus 91-09', 'line:IDFM:C01571': 'Bus 91-10', 'line:IDFM:C02201': 'Bus 91-11', 'line:IDFM:C01423': 'Bus 1', 'line:IDFM:C01426': 'Bus 2', 'line:IDFM:C02129': 'Bus 3', 'line:IDFM:C02130': 'Bus 3s', 'line:IDFM:C02137': 'Bus 4', 'line:IDFM:C02138': 'Bus 4s', 'line:IDFM:C01427': 'Bus 5', 'line:IDFM:C02131': 'Bus 6', 'line:IDFM:C02136': 'Bus 7', 'line:IDFM:C02135': 'Bus 7s', 'line:IDFM:C01717': 'Bus 8', 'line:IDFM:C02139': 'Bus 8s', 'line:IDFM:C02133': 'Bus 9', 'line:IDFM:C02132': 'Bus 9s', 'line:IDFM:C00324': 'Bus 1', 'line:IDFM:C00292': 'Bus 1', 'line:IDFM:C00294': 'Bus 2', 'line:IDFM:C01473': 'Bus 2', '

,reference,publication_start_date,publication_end_date,pt_object_type,pt_object_uri,pt_object_name,severity,titre,moteur,notification,application_start_date,application_end_date,created_at,class,line_name
32115,sncf_PCNV0beaffe2-79fc-11eb-9b19-005056a44da2:1,2021-02-28 20:34:51+01:00,2021-02-28 22:30:42+01:00,line,line:0:800:H,Unable to find object,perturbée,Ligne H: fortement ralenti,Trafic fortement ralenti sur la ligne H. Motif...,Ligne H: fortement ralenti,2021-02-28 20:30:00+01:00,2021-02-28 22:30:00+01:00,2021-02-28 21:45:07+01:00,Personnes sur les voies,NaN
37960,sncf_PCNV3d08685c-1336-11ec-8af2-005056a47b86:1,2021-09-11 21:26:56+02:00,2021-09-11 22:50:00+02:00,line,line:IDFM:C01739,J,perturbée,Ligne J: Paris St-Lazare - Maisons Laffit. ral...,Le trafic est ralenti entre Paris Saint-Lazare...,Ligne J: Paris St-Lazare - Maisons Laffit. ral...,2021-09-11 21:21:00+02:00,2021-09-11 22:20:00+02:00,2021-09-11 21:30:06+02:00,Voyageur,Train Transilien J
41567,sncf_PCNV58c8d410-54fe-11eb-9c95-005056a47b86:1,2021-01-12 18:48:12+01:00,2021-01-12 22:00:00+01:00,line,line:0:800:P,Unable to find object,perturbée,Ligne P : Château-Thierry - Meaux interrompu,Le trafic est interrompu de Château-Thierry ve...,Ligne P : Château-Thierry - Meaux interrompu,2021-01-12 18:30:00+01:00,2021-01-12 21:30:00+01:00,2021-01-12 20:30:05+01:00,Electrique,NaN
249,10_TICE-20210701113549,2021-07-01 11:30:00+02:00,2021-07-01 21:00:00+02:00,line,line:0:400400405:405,Unable to find object,perturbée,Ligne 405 : Organisation Estivale à Corbeil-Es...,En raison d'une organisation Estivale l'arrêt ...,NaN,2021-07-01 11:30:00+02:00,2021-07-01 21:00:00+02:00,2021-07-01 11:36:23+02:00,NaN,NaN
41464,sncf_PCNV5839f08c-64b2-11ec-b89f-005056a47b86,2021-12-24 13:09:26+01:00,2021-12-24 14:30:05+01:00,line,line:IDFM:C01731,R,perturbée,Ligne R : Paris Lyon - Montereau différé,Le train suivant voit son départ différé de 40...,Ligne R : Paris Lyon - Montereau différé,2021-12-24 13:05:00+01:00,2021-12-24 14:30:00+01:00,2021-12-24 13:10:03+01:00,Exploitation,Train Transilien R
2895,keolis4-20211118063456,2021-11-18 06:30:00+01:00,2021-11-18 19:00:00+01:00,line,line:IDFM:C00003,Licorne,perturbée,Bus Licorne : perturbations,Bus Licorne : En raison d'un incident d'exploi...,Bus Licorne : perturbations,2021-11-18 06:30:00+01:00,2021-11-18 19:00:00+01:00,2021-11-18 06:37:16+01:00,Exploitation,Bus Licorne
31119,sncf_PCNV03e72084-5baf-11eb-b9c8-005056a44da2:1,2021-01-21 07:07:58+01:00,2021-01-21 08:25:00+01:00,line,line:0:800:C,Unable to find object,perturbée,Ligne C Dourdan - Brétigny : suppression,Train supprimé :- Le train KYVI prévu au dépar...,Ligne C Dourdan - Brétigny : suppression,2021-01-21 06:57:00+01:00,2021-01-21 07:55:00+01:00,2021-01-21 07:10:13+01:00,Obstacle voie,NaN
39910,sncf_PCNV4c859850-9e7d-11eb-89cb-005056a47b86:1,2021-04-16 08:30:52+02:00,2021-04-16 12:30:00+02:00,line,line:0:800:D,Unable to find object,perturbée,Ligne D : Villeneuve St-G - Orry la Ville ralenti,Le trafic est ralenti entre Villeneuve-Saint-G...,NaN,2021-04-16 08:24:00+02:00,2021-04-16 12:00:00+02:00,2021-04-16 08:45:07+02:00,Incendie,NaN
55765,sncf_PCNVbf2dafa6-4c8c-11ec-a228-005056a47b86:1,2021-11-23 19:21:49+01:00,2021-11-23 21:30:00+01:00,line,line:IDFM:C01728,D,perturbée,Ligne D : Paris Lyon - Paris Nord interrompu,Le trafic est interrompu de Paris Gare de Lyon...,Ligne D : Paris Lyon - Paris Nord interrompu,2021-11-23 19:15:00+01:00,2021-11-23 20:00:00+01:00,2021-11-23 19:45:04+01:00,Bagage oublié,RER D
42950,sncf_PCNV63b8f386-177d-11ec-b850-005056a44da2:1,2021-09-17 07:54:14+02:00,2021-09-17 09:00:16+02:00,line,line:IDFM:C01742,A,perturbée,Ligne A : perturbé,Le trafic est perturbé sur l'ensemble de la li...,NaN,2021-09-17 08:04:00+02:00,2021-09-17 09:00:00+02:00,2021-09-17 08:10:21+02:00,Bagage oublié,RER A


In [278]:
px.bar(chaos_classified.line_name.value_counts())

In [279]:
stats_byline_byclass = chaos_classified.groupby(["line_name", "class"]).agg({"reference": "count"}).reset_index()
px.bar(stats_byline_byclass, x="line_name", y="reference", color="class",
       category_orders={'line_name': chaos_classified.line_name.value_counts().index.to_list()})

In [280]:
chaos_classified.to_csv("chaos_classified.csv")